In [1]:
from inference import OllamaClient
client = OllamaClient(base_url="http://localhost:11434")
model="gemma3:4b"
client.download_model(model=model)
models = client.list_models()
print(models)
client.warmup_model(model=model)
# client.warmup_model(model=evaluator_model)

Model gemma3:4b downloaded successfully.
[('gemma3:4b', '4.3B')]
Model gemma3:4b warmed up successfully.


In [3]:
prompt="What is the meaning of Cheese?"
res = client.generate(model=model, prompt=prompt, temperature=0.7, max_tokens=50)
print(res)
# res = client.generate(model=evaluator_model, prompt=prompt, temperature=0.7, max_tokens=50)
# print(res)

{"model":"gemma3:4b","created_at":"2025-11-12T20:35:09.94511364Z","response":"Okay, let's break down the meaning of \"cheese\" – it's a surprisingly complex one, encompassing a lot more than just a dairy product! Here's a breakdown of its meanings, categorized for clarity:\n\n**1.","done":true,"done_reason":"length","context":[105,2364,107,3689,563,506,6590,529,57626,236881,106,107,105,4368,107,19058,236764,1531,236789,236751,2541,1679,506,6590,529,623,120563,236775,1271,625,236789,236751,496,33887,3996,886,236764,97749,496,2606,919,1082,1164,496,26283,1698,236888,5715,236789,236751,496,25890,529,1061,47670,236764,74252,573,29972,236787,108,1018,236770,236761],"total_duration":4978521254,"load_duration":514796821,"prompt_eval_count":16,"prompt_eval_duration":134678244,"eval_count":50,"eval_duration":4244849380}
Okay, let's break down the meaning of "cheese" – it's a surprisingly complex one, encompassing a lot more than just a dairy product! Here's a breakdown of its meanings, categori

## Prepare PRewriteDataset and trainer

The cells below show how to wrap the loaded DataFrame into `PRewriteDataset`,
create an Ollama evaluator and the `PRewritePPOTrainer`, and run a short
training run for testing. Adjust `max_steps` for real training.

In [4]:
# Create PRewriteDataset from the already-loaded DataFrame `df`
from genericpath import exists
import pandas as pd
import pickle
from load_datasets import PRewriteDataset
if exists("data/gsm8k_train.pkl"):
    df = pickle.load(open("data/gsm8k_train.pkl", "rb"))
else:
    splits = {'train': 'main/train-00000-of-00001.parquet', 'test': 'main/test-00000-of-00001.parquet'}
    df = pd.read_parquet("hf://datasets/openai/gsm8k/" + splits["train"])
    pickle.dump(df, open("data/gsm8k_train.pkl", "wb"))
# reserve 10% as dev (paper used 10% for GSM8K)
ds = PRewriteDataset(df, split="train", dev_fraction=0.1, seed=42)
print('train size:', len(ds.get_split_df('train')) )
if ds.get_split_df('dev') is not None:
    print('dev size:', len(ds.get_split_df('dev')))

train size: 6726
dev size: 747


In [ ]:
# Load finetune model from Hugging Face (will save locally if not present)
from finetune import load_from_hf
import torch
torch.cuda.is_available()


#model_id = "Qwen/Qwen3-4B-Instruct-2507"
model_id = "Qwen/Qwen2.5-1.5B-Instruct"
save_dir = f"models/{model_id}-local"

# This will try to load from `save_dir` first; if missing it downloads and saves there.
tokenizer, hf_model = load_from_hf(model_id, save_to=save_dir, trust_remote_code=True)
print('Model/tokenizer available at:', save_dir)

# Expose the local path for the trainer to use as `base_model` if desired
base_model_local = save_dir


In [ ]:
#print full head, not cut off
pd.set_option('display.max_colwidth', None)
ds.get_split_df('train').head()

,question,answer
0,"Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72
1,"Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?","Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10"
2,"Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?","In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.\nBetty's grandparents gave her 15 * 2 = $<<15*2=30>>30.\nThis means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.\n#### 5"
3,"Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?","Maila read 12 x 2 = <<12*2=24>>24 pages today.\nSo she was able to read a total of 12 + 24 = <<12+24=36>>36 pages since yesterday.\nThere are 120 - 36 = <<120-36=84>>84 pages left to be read.\nSince she wants to read half of the remaining pages tomorrow, then she should read 84/2 = <<84/2=42>>42 pages.\n#### 42"
4,James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year?,He writes each friend 3*2=<<3*2=6>>6 pages a week\nSo he writes 6*2=<<6*2=12>>12 pages every week\nThat means he writes 12*52=<<12*52=624>>624 pages a year\n#### 624
